<a href="https://colab.research.google.com/github/K-myeongji/kakaotalk/blob/preprocessing/%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 전처리

In [ ]:
import re
import os
import pandas as pd

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## 사용하면서 수정하는 부분(이외에는 수정x) ----------------------------------------

name = '**' # 카카오톡 내에서의 이름
file_name = '***' # 파일로 저장할 이름(풀네임) -> 개인톡일 경우 이름_개인 / 단체톡일 경우 이름_단체  *** 중요

# ---- 카카오톡 파일 경로 및 이름
f_path = '*' # 처음 카톡 파일 불러오기
f_name_1 = '*' # 채팅방 여러개일 경우 합치는  --> 하나면 그냥 f_name_1만 쓰고, f_name_2는 주석처리
# f_name_2 = '*'  # 채팅방 여러개일 경우 합치는
down_path = '*' # 전처리한 카톡 데이터 저장

# ---- train 파일 경로 및 이름
train_path = '*'

# ---- 설문 답변 경로 및 이름
survey_path = '*'
survey_name = '*'

# ---- 마지막 결과 파일 저장 경로(수정x - 우리 공유폴더 안에 저장한 거)
final_path = '*/'
final_name = '*'

## ---------------------------------------------------------------------------------

## 1) 데이터 불러오기

In [ ]:
def katalk_msg_parse(file_path):
    my_katalk_data = list()
    katalk_msg_pattern = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2},.*:"  # 카카오톡 스마트폰 버전
    #katalk_msg_pattern = "\[(.*?)\] \[(오\S) ([0-9]{1,2}:[0-9]{1,2})\] (.*)"  # 카카오톡 PC버전
    date_info = "[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일"
    in_out_info = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2}:.*"

    for line in open(file_path):
        if re.match(date_info, line) or re.match(in_out_info, line):
            continue
        elif line == '\n':
            continue
        elif re.match(katalk_msg_pattern, line):
            line = line.split(",")
            date_time = line[0]
            user_text = line[1].split(" : ", maxsplit=1)
            user_name = user_text[0].strip()
            text = user_text[1].strip()
            my_katalk_data.append({'date_time': date_time,
                                   'user_name': user_name,
                                   'text': text
                                   })

        else:
            if len(my_katalk_data) > 0:
                my_katalk_data[-1]['text'] += "\n"+line.strip()

    my_katalk_df = pd.DataFrame(my_katalk_data)

    return my_katalk_df


## https://regexr.com/

In [ ]:
import random
import string

df = katalk_msg_parse(os.path.join(f_path, f_name_1))

# # 익명화 함수 정의
# def anonymize_name(name):
#     # 이름을 익명화하여 반환하는 함수
#     random_chars = ''.join(random.choices(string.ascii_lowercase, k=2))
#     random_nums = ''.join(random.choices(string.digits, k=2))
#     anonymized_name = random_chars + random_nums
#     return anonymized_name

# # 중복된 user_name이 있는 경우 동일한 익명화된 아이디를 사용하도록 처리
# df['user_name'] = df.groupby('user_name')['user_name'].transform(lambda x: anonymize_name(x.iloc[0]))

# f_name_2 파일이 존재하는 경우에만 df_2로 데이터프레임 생성
df['date_time'] = df['date_time'].str.replace('오전', 'AM')
df['date_time'] = df['date_time'].str.replace('오후', 'PM')
# 휴대폰 기종에 따라 datetime 형식 변환
try:
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y년 %m월 %d일 %p %I:%M')  # 예진주영명지 - 갤럭시용
except ValueError:
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y. %m. %d. %p %I:%M')  # 효림 - 아이폰용

df = df.reset_index(drop=True)
print('파일이 하나만 들어왔고, 그 파일을 데이터프레임 형식으로 불러왔습니다.')

df



In [ ]:
import re

# 정규표현식 패턴을 사용하여 날짜 형식의 문자열을 찾아 제거하는 함수
def remove_date_string(text):
    pattern = r'\d+년 \d+월 \d+일 (오전|오후) \d+:\d+'
    return re.sub(pattern, '', text)

# 'text' 열에 적용하여 날짜 형식의 문자열을 제거하고 새로운 열 'clean_text'를 생성
df['text'] = df['text'].apply(remove_date_string)

df

#### 중간에 생략한 코드 있음

In [ ]:
#시간 전처리
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['weekday'] = df['date_time'].dt.day_name()
df['hour'] = df['date_time'].dt.hour


df['minute'] = df['date_time'].dt.minute
df['second'] = df['date_time'].dt.second

df

In [ ]:
# 날짜 가르기
start_date = '2023-01-01 00:00:00'
end_date = '2023-06-01 00:00:00'

mask = (df['date_time'] >= start_date) & (df['date_time'] <= end_date)
df_sep = df.loc[mask]

df_sep = df_sep.reset_index(drop=True)
df_sep

## 2) 텍스트 제외 데이터 전처리


In [ ]:
# # 문장 길이 및 단어 수 칼럼 추가 -- 맞아?? 효림
df_sep['msg_len'] = df_sep['text'].str.len()
df_sep['msg_word_count'] = df_sep['text'].str.split().str.len()

In [ ]:
#동영상, 사진 등 전처리 후 제거

audio_visual_text = '^동영상$|^사진$|^사진 [0-9]{1,2}장$'
mask = df_sep['text'].str.contains(audio_visual_text)

# 동영상, 사진 관련 덱스트가 있는 행은 1로 표시, 그렇지 않은 행은 0으로 표시
df_sep.loc[mask, 'audio_visual'] = 1
df_sep.loc[~mask, 'audio_visual'] = 0

df_sep.loc[mask, 'msg_len'] = 0
df_sep.loc[mask, 'msg_word_count'] = 0
print('동영상 및 사진 데이터 전처리 전: ', len(df_sep))

df_sep[df_sep['audio_visual']==1]
df_sep = df_sep[df_sep['audio_visual'] != 1]
print('동영상 및 사진 데이터 전처리 후: ', len(df_sep))

df_sep = df_sep.reset_index(drop=True)
df_sep

In [ ]:
# --- 보이스톡, 페이스톡, 이모티콘, url, 정산 메시지 제거

!pip install emoji~=1.6.3 # 이모티콘 전처리를 위한 패키지 설치
!pip install urlextract # url제거를 위한 패키지 설치

In [ ]:
# 보이스톡, 페이스톡 등 전처리 후 제거 ---- 수정: 전처리 전 데이터프레임 df가 오길래 전처리 후 데이터프레임 df_sep으로 -- 효림
df_sep[~df_sep['text'].str.contains('보이스톡|페이스톡')]
print('보이스톡, 페이스톡 데이터 전처리 후: ', len(df_sep))

In [ ]:

from urlextract import URLExtract

text = """Google Colab: https://colab.research.google.com.
       Introduction for google colab: https://colab.research.google.com/notebooks/intro.ipynb"""

extractor = URLExtract()
urls = extractor.find_urls(text)

# for url in urls:
#     print(url)

extractor = URLExtract()
df_sep['url'] = df_sep['text'].apply(extractor.find_urls)

df_sep['url_count'] = df_sep['url'].apply(len)
url_df = df_sep.loc[df_sep['url_count'] > 0]
url_df[['date_time','user_name','url','url_count']]

df_sep = df_sep[df_sep['url_count'] == 0]
print('url 데이터 전처리 후: ', len(df_sep))
df_sep

In [ ]:

import emoji

def extract_emojis(text):
    emoji_list = list()
    for c in text:
        if c in emoji.UNICODE_EMOJI['en']:
            emoji_list.append(c)
    return emoji_list

mimetic = "[laughter!?~]+"
punctuations = "[,.]{2,}"
emo_type1_facial1 = "[;:]{1}[\^\'-]?[)(DPpboOX]"
emo_type1_facial2 = "[>ㅜㅠㅡ@\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@<];*"
emo_type3 = "\(.+?\)"

nonverbal_list = [mimetic, emo_type1_facial1, emo_type1_facial2, emo_type3]

df_sep['nonverbal'] = df_sep['text'].str.findall('|'.join(nonverbal_list)) + df_sep['text'].map(extract_emojis)
df_sep['nonverbal_count'] = df_sep['nonverbal'].apply(len)

df_sep = df_sep[df_sep['nonverbal_count'] == 0] # 이모지가 포함된 문장에서 이모지만 삭제해주는
df_sep = df_sep[~(df_sep['text'] == '이모티콘')]
df_sep['text'] = df_sep['text'].str.replace('이모티콘', '')

print('이모티콘 데이터 전처리 후: ', len(df_sep))
df_sep

In [ ]:
# 정산 메시지 제거
df_sep = df_sep[~df_sep['text'].str.contains('정산하기를 요청했어요.')]
df_sep = df_sep[~df_sep['text'].str.contains('받은 카카오페이머니는 송금 및 온/오프라인 결제도 가능해요.')]
df_sep['text'] = df_sep['text'].str.replace(r'(?:\b(?:1|2|3|4|5|6|7|8|9|[1-9][0-9]|100)\b\s*)?송금 받기 전까지 내역 상세화면에서 취소할 수 있어요\.', '', regex=True)
print('정산 메시지 데이터 전처리 후: ', len(df_sep))

df_sep = df_sep[df_sep['text'].str.strip().astype(bool)] ### 임의로 추가(text가 비어있는 열 제외하는 코드)
df_sep = df_sep.reset_index(drop=True)
df_sep